## Getting started with LoRaWAN on IoT-LAB using RIOT

The goal of this notebook is to discover the basics of LoRaWAN communication using RIOT on IoT-LAB and TheThingsNetwork (TTN) as LoRaWAN network provider.

> We recommend that you follow the [LoRa boards](../lora-boards/lora-boards.ipynb) notebook if not done already.

This Notebook is divided in several steps:
1. The first step requires you to configure a LoRaWAN application with one device on the TTN backend
2. Then you will submit an experiment with one ST B-L072Z-LRWAN1 node (known as st-lrwan1) on the Saclay site
3. Once the experiment is running, you will build and flash a RIOT application that provides a shell to control the LoRaWAN stack. On the board, you will configure the identifiers and keys required for Over-The-Air Activation (OTAA).
4. In the last step of this tutorial, you will exchange messages between the board and TTN.

### LoRaWAN overview

A LoRaWAN network includes a set of subsystems and protocols allowing _end-devices_ to communicate on long distances while consuming little energy.

To communicate by radio, LoRaWAN devices and gateways use the LoRa radio technology. This technology is based on a Chirp Spreading Spectrum frequency modulation within public radio frequency bands (ISM). The access to the physical layer is regulated and depends on the region of use.

For example, LoRaWAN defines the notion of "datarate". A datarate index is the combination of a spreading factor (SF) and a bandwidth (BW), as shown in the table below:

<figure style="text-align:center">
    <img src="images/lorawan-datarate-table.png" width="650" align="center"/><br/><br/>
    <figcaption><em>LoRaWAN Datarate table for EU868 band</em></figcaption>
</figure>

A datarate of 0 uses a spreading factor of 12 and a bandwidth of 125kHz, in this case the transmission is the slowest (250 bit/s). A datarate of 6 uses a spreading factor of 7 and a bandwidth of 250kHz, the transmission is the fastest in this case (11kbit/s).

You can find more information on LoRaWAN [here](https://fr.wikipedia.org/wiki/LoRaWAN) and [here](https://docs.wixstatic.com/ugd/eccc1a_20fe760334f84a9788c5b11820281bd0.pdf):

On IoT-LAB, the EU868 ISM band is used by the boards, because they are all located in Europe and the boards can only transmit on this band.

<figure style="text-align:center">
    <img src="images/lorawan-classes.jpg" width="640" align="center"/><br/><br/>
    <figcaption><em>The LoRaWAN stack</em></figcaption>
</figure>

The LoRaWAN specifications define 3 classes of end-devices: A, B, C. Each of which has specific access to the physical layer. For more details, please refer to the LoRa specifications provided by the LoRa Alliance [here](https://www.lora-alliance.org/lorawan-for-developers).

Here is a global overview of a LoRaWAN network:

<figure style="text-align:center">
    <img src="images/lorawan-system.png" width="640" align="center"/><br/><br/>
    <figcaption><em>The LoRaWAN infrastructure</em></figcaption>
</figure>

End devices use LoRa modulation to communicate with the gateways, which themselves use the regular Internet protocols to communicate with the LoRaWAN server, owned by a provider.

### TTN overview

TheThingsNetwork is a LoRaWAN network provider which philosophy is oriented towards open access to IoT networks:

- the network is built by users: anyone can install a gateway connected to the TTN backend and thus increase the global coverage of the network
- access to the TTN network is free of charge, without restriction
- the TTN ecosystem is very dynamic, especially in Europe (about 3000 gateways already deployed and more than 17,000 applications)

A TTN gateway is installed in the IoT-LAB Saclay site and connected to the TTN backend server. Thus, this allows LoRa devices deployed in the Saclay site to communicate with the TTN backend.

### Configure a TTN application

Before using the LoRa devices in IoT-LAB, you have to create an account and register an application with a device configured on TTN.

1. If you don’t already have a TTN account, create one from [here](https://account.thethingsnetwork.org/register)

<figure style="text-align:center">
    <img src="images/ttn-signup.png" width="640" align="center"/><br/><br/>
    <figcaption><em>TTN signup interface</em></figcaption>
</figure>

After submitting the account creation form, check your emails and then activate the account.

2. Add an application to your TTN account by following [the official documentation](https://www.thethingsnetwork.org/docs/applications/add.html).

Each end-device of a LoRaWAN network is part of a LoRaWAN application. You have to configure a first application to allow you to communicate with your device over LoRa.

3. Register a device in your TTN application by following [this documentation](https://www.thethingsnetwork.org/docs/devices/registration.html).
   Keep the default Other-The-Air Activation (OTAA) procedure.The OTAA activation requires 3 parameters:
   - **Device EUI**: the device unique identifier is a 8 bytes array (16 hex char string)
   - **Application EUI**: the application unique identifier is a 8 bytes array (16 hex char string)
   - **Application Key**: the application key is a 16 bytes array (32 hex char string)

<figure style="text-align:center">
    <img src="images/ttn-register-device.png" width="500" align="center"/>
    <img src="images/ttn-show-device.png" width="450" align="center"/><br/><br/>
    <figcaption><em>LoRaWAN device registration and configuration on TTN</em></figcaption>
</figure>

**Important:** In the following instructions of this tutorial, we consider that your application name is **_iotlab-lorawan_** and your node id is **_iotlab-node_**. 

**Replace these names with the information you configured in your TTN account**.

### Start an experiment with one LoRa device

Now let's book one the LoRa device available in the IoT-LAB testbed. The boards are [ST B-L072Z-LRWAN1](https://www.st.com/en/evaluation-tools/b-l072z-lrwan1.html):

<figure style="text-align:center">
    <img src="images/en.b-l072z-lrwan1.jpg" width="320" align="center"/><br/><br/>
    <figcaption><em>The ST B-L072Z-LRWAN1 board used in IoT-LAB</em></figcaption>
</figure>

1. Submit an experiment with one LoRa device on IoT-LAB:

In [ ]:
!iotlab-experiment submit -n "ttn-getting-started" -d 120 -l 1,archi=st-lrwan1:sx1276+site=saclay

2. Wait for the experiment to be in the "Running" state:

In [ ]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later.

### Build and flash the RIOT firmware

To use the ST board with the LoRaWAN TTN network, you must first flash the RIOT LoRaWAN test application, `tests/pkg_semtech-loramac`:

1. Build and flash the `tests/pkg_semtech-loramac` application of RIOT.

This application provides a shell to finely control the loramac stack on the device. By default, the target of the application corresponds to the `st-lrwan1` board type in IoT-LAB.

In [ ]:
!make IOTLAB_NODE=auto-ssh -C ../../RIOT/tests/pkg_semtech-loramac flash

To communicate in LoRaWAN, this application relies on a library (a "package") developed by Semtech and [available on GitHub](https://github.com/Lora-net/LoRaMac-node). As of today, this library is the reference implementation of the MAC layer of the LoRaWAN protocol for end-devices.

2. Open a terminal: `File > New > Terminal` and connect to the shell by running the following command in the terminal:

You can press *Enter* to get the `>` prompt.

You are now ready for configuring your LoRaWAN device!

### Configure the LoRaWAN device

The RIOT shell provides the `loramac` command to control the loramac stack, join the network, and send packets:

1. In the RIOT shell, let's start by configuring the loramac stack with your DEVEUI, APPEUI and APPKEY:

 >**note:** Use the values of your registered device in the TTN backend.

2. Configure a fast datarate, e.g. 5, corresponding to a bandwidth of 125kHz and a spreading factor of 7, since the nodes are very close to the gateway:

3. Now that the device is correctly configured for OTAA activation, it is time to join it to the network:

> **Reminder:** The principle of OTAA activation is to use the identifiers of the device (Device EUI) and the application (Application EUI) with the application key to build a secure connection request.
> Upon receipt of this connection request, the LoRaWAN server checks that the message is valid, using the Message Integrity Code (MIC), and it verifies the validity of the identifiers.
> If the request is valid, the server responds with an acknowledgement message containing a "nonce" (an arbitrary number) that will be used by the server and by the device to determine the session keys (network and application) from the initial application key.

On the TTN web console, go to the **Data** tab of the `iotlab-node` page. You should see the activation message received by the TTN backend (the item is clickable).

4. Now use the `tx` subcommand to send a payload to the backend: 

Still in the **Data** tab of the `iotlab-node` web page, you should see the message received by the TTN backend:

<figure style="text-align:center">
    <img src="images/ttn-tx-device.png" width="640" align="center"/><br/><br/>
    <figcaption><em>Messages received on TTN</em></figcaption>
</figure>

Payload are encoded in hexadecimal values: one ascii byte => 2 hexadecimal characters. You can decode it using the following function (replace the value stored in `hex_str` with the payload received) to verify that the message is correct:

In [ ]:
import codecs
hex_str = "4c6f526157414e40496f542d4c4142"
print(codecs.decode(hex_str, "hex"))

5. Send a downlink message from the **Overview** tab of the `iotlab-node` (for example use the following hexadecimal values: **48656C6C6F52494F5421**)

**Note**: Nothing is received by the node because a LoRaWAN class A device (the default for a RIOT LoRaWAN node) only have a short RX window after a send.

In order to receive the downlink message, just send another message to TTN: 

You sent and received LoRaWAN messages using RIOT on IoT-LAB, congratulations !

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

The serial link connection through SSH will be closed automatically.